### Funkcje bo charted nie działa

In [1]:
import geopandas as gpd
from  imgw_static import get_map_zlewnie, get_static_data_hydro,get_static_data_meteo
import pandas as pd
def get_hydro_metadata() -> gpd.GeoDataFrame:
    """
    Wczytuje metadane hydrologiczne ze wskazanego URL (np. API IMGW)
    i zwraca je jako GeoDataFrame w układzie EPSG:2180 (PUWG 1992).
    """
    hydro_url = "http://danepubliczne.imgw.pl/api/data/hydro2/"
    hydro_stat = pd.read_json(hydro_url)
    df = hydro_stat.iloc[:, 0:4]
    
    # Tworzenie geometrii
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    )
    gdf.columns = ['Station Code','Station Name','Lon','Lat','geometry']
    # Konwersja do układu PL (PUWG 1992)
    gdf = gdf.to_crs("EPSG:2180")
    
    return gdf
def get_meteo_metadata() -> gpd.GeoDataFrame:
    """
    Wczytuje metadane meteorologiczne ze wskazanego URL (np. API IMGW)
    i zwraca je jako GeoDataFrame w układzie EPSG:2180 (PUWG 1992).
    """
    meteo_url = "http://danepubliczne.imgw.pl/api/data/meteo/"
    meteo_stat = pd.read_json(meteo_url)
    df = meteo_stat.iloc[:, 0:4]
    
    # Tworzenie geometrii
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326"
    )
    gdf.columns = ['Station Code','Station Name','Lon','Lat','geometry']
    # Konwersja do układu PL (PUWG 1992)
    gdf = gdf.to_crs("EPSG:2180")
    
    return gdf

# Preprocessing

## Dane Hydrologiczne

In [2]:
import pandas as pd

df = get_static_data_hydro()

Dane Pomiarowe

In [3]:
print("Shape: ", df.shape)
print("Ilość stacji hydro w danych pomiarowych: ",df["Station Code"].nunique())
df.head()


Shape:  (4273340, 10)
Ilość stacji hydro w danych pomiarowych:  917


,Station Code,Station Name,Name,Hydro Year,Hydro Month,Day,Water Level,Flow,Water Temp,Calendar Month
0,149180020,CHA£UPKI,Odra (1),2010,1,1,158,30.8,99.9,11
1,149180020,CHA£UPKI,Odra (1),2010,1,2,154,28.5,99.9,11
2,149180020,CHA£UPKI,Odra (1),2010,1,3,150,26.0,99.9,11
3,149180020,CHA£UPKI,Odra (1),2010,1,4,149,25.4,99.9,11
4,149180020,CHA£UPKI,Odra (1),2010,1,5,149,25.9,99.9,11


Dane Meta

In [4]:
gdf = get_hydro_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji hydro w meta danych: ",gdf["Station Code"].nunique())
gdf.head()

Shape:  (859, 5)
Ilość stacji hydro w meta danych:  859


,Station Code,Station Name,Lon,Lat,geometry
0,150160330,SZCZYTNA,16.443056,50.415556,POINT (318401.478 286283.506)
1,150160340,SARNY,16.465833,50.547778,POINT (320520.661 300923.221)
2,150160350,SZALEJÓW GÓRNY,16.537222,50.418333,POINT (325098.752 286366.241)
3,150160360,STARKÓW,16.580000,50.377500,POINT (327988.637 281728.229)
4,150160370,TOPOLICE,16.609167,50.366944,POINT (330023.848 280487.958)


## Wstępna selekcja danych hydrologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [5]:
print("Ile stacji rozpoczęło pomiary przed lub w 2010 roku:",(df.groupby("Station Code")["Hydro Year"].min() == 2010).mean() * 100,"%")

Ile stacji rozpoczęło pomiary przed lub w 2010 roku: 92.80261723009815 %


Odcinamy stacje z pierszwego warunku.

In [6]:
valid_stations = df.groupby("Station Code")["Hydro Year"].min()
valid_stations = valid_stations[valid_stations == 2010].index

# Step 2: Filter the DataFrame to include only those stations
filtered_df = df[df["Station Code"].isin(valid_stations)]

Odcinamy stacje z drugiego warunku.

In [7]:
valid_stations = gdf['Station Code']

# Step 3: Filter the DataFrame to include only those stations
filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [8]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych hydro danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (3533386, 10)
Ilość stacji w odfiltrowanych hydro danych:  697


Eksportujemy odfiltrowane dane do static_data

W efekcie mamy spójne dane zaczynające się od 2010 i z lokalizacją

### Mapowanie Kodów 

Stan wody 9999 oznacza brak danych w bazie.

Przepływ 99999.999 oznacza, że przepływ w tym dniu nie był opracowywany.

Temperatura 99.9 oznacza brak danych w bazie, która może wynikać np. z braku pomiarów temperatury na stacji.

In [9]:
import numpy as np

filtered_df_2.replace({
    'Water Level': {9999: np.nan},
    'Flow': {99999.999: np.nan},
    'Water Temp': {99.9: np.nan}
}, inplace=True)

C:\Users\Tymek\AppData\Local\Temp\ipykernel_7976\3020362930.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2.replace({


### Dodanie Daty Kalendarzowej - DEMO Solution

In [14]:

from datetime import datetime
def convert_to_calendar_date(row):
    hydro_year = row['Hydro Year']
    hydro_month = row['Hydro Month']
    hydro_day = row['Day']
    
    # Miesiące 1 i 2 to listopad i grudzień poprzedniego roku
    if hydro_month == 1:
        month = 11
        year = hydro_year - 1
    elif hydro_month == 2:
        month = 12
        year = hydro_year - 1
    else:
        month = hydro_month - 2  # 3 => styczeń
        year = hydro_year

    try:
        return datetime(year, month, hydro_day)
    except ValueError:
        return pd.NaT  # Obsługa błędnych dat

In [15]:
filtered_df_2["Calendar Date"] = filtered_df_2.apply(convert_to_calendar_date,axis=1)

C:\Users\Tymek\AppData\Local\Temp\ipykernel_7976\3658452928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2["Calendar Date"] = filtered_df_2.apply(convert_to_calendar_date,axis=1)


In [16]:
filtered_df_2.to_parquet("../static_data/filtered_hydro_data.parquet.gzip")

## Dane Opad

In [32]:
df = pd.read_parquet('../static_data/meteo_opad_data.parquet.gzip')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1801634 entries, 0 to 13810
Data columns (total 16 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Station Code             int64  
 1   Station Name             object 
 2   Year                     int64  
 3   Month                    int64  
 4   Day                      int64  
 5   Daily Precip Sum         float64
 6   SMDB Measurement Status  float64
 7   Precipitation Type       object 
 8   Snow Cover Height        int64  
 9   PKSN Measurement Status  float64
 10  Fresh Snow Height        int64  
 11  HSS Measurement Status   float64
 12  Snow Type                float64
 13  GATS Measurement Status  float64
 14  Snow Cover Type          object 
 15  RPSN Measurement Status  float64
dtypes: float64(7), int64(6), object(3)
memory usage: 233.7+ MB


In [34]:
df.isna().mean()

Station Code               0.000000
Station Name               0.000000
Year                       0.000000
Month                      0.000000
Day                        0.000000
Daily Precip Sum           0.000000
SMDB Measurement Status    0.895250
Precipitation Type         0.104965
Snow Cover Height          0.000000
PKSN Measurement Status    0.211784
Fresh Snow Height          0.000000
HSS Measurement Status     0.108527
Snow Type                  0.794094
GATS Measurement Status    0.206453
Snow Cover Type            0.368733
RPSN Measurement Status    0.900739
dtype: float64

In [35]:
print("Shape: ", df.shape)
print("Ilość stacji hydro w danych pomiarowych: ",df["Station Code"].nunique())
df.head()

Shape:  (1801634, 16)
Ilość stacji hydro w danych pomiarowych:  1137


,Station Code,Station Name,Year,Month,Day,Daily Precip Sum,SMDB Measurement Status,Precipitation Type,Snow Cover Height,PKSN Measurement Status,Fresh Snow Height,HSS Measurement Status,Snow Type,GATS Measurement Status,Snow Cover Type,RPSN Measurement Status
0,249180020,WARSZOWICE,2010,1,1,1.1,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0
1,249180020,WARSZOWICE,2010,1,2,5.1,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0
2,249180020,WARSZOWICE,2010,1,3,0.6,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0
3,249180020,WARSZOWICE,2010,1,4,0.3,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0
4,249180020,WARSZOWICE,2010,1,5,3.3,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0


In [36]:
print("Ile stacji rozpoczęło pomiary przed lub w 2010 roku:",(df.groupby("Station Code")["Year"].min() == 2010).mean() * 100,"%")

Ile stacji rozpoczęło pomiary przed lub w 2010 roku: 84.25681618293756 %


In [37]:
gdf = get_meteo_metadata()
print("Shape: ", gdf.shape)
print("Ilość stacji hydro w meta danych: ",gdf["Station Code"].nunique())
gdf.head()

Shape:  (766, 5)
Ilość stacji hydro w meta danych:  766


,Station Code,Station Name,Lon,Lat,geometry
0,252210290,RYBIENKO,21.429167,52.577778,POINT (664545.901 526326.119)
1,351230497,WŁODAWA,23.529444,51.553333,POINT (813865.629 419371.773)
2,352220385,SIEDLCE,22.244722,52.181111,POINT (721755.946 484409.713)
3,250240010,STRZYŻÓW,24.035556,50.840000,POINT (854357.945 342430.384)
4,352230399,TERESPOL,23.621944,52.078611,POINT (816563.802 478131.955)


## Wstępna selekcja danych meteorologicznych

Wybieramy stacje które:

1. Rozpoczęły pomiary przed lub w 2010 roku.
2. Można zmapować do ich lokalizacji.

In [38]:
valid_stations = df.groupby("Station Code")["Year"].min()
valid_stations = valid_stations[valid_stations == 2010].index

# Step 2: Filter the DataFrame to include only those stations
filtered_df = df[df["Station Code"].isin(valid_stations)]

In [39]:
print("Ilość stacji w odfiltrowanych (filtracja czasowa) meteo danych: ",filtered_df["Station Code"].nunique())

Ilość stacji w odfiltrowanych (filtracja czasowa) meteo danych:  958


In [40]:
valid_stations = gdf['Station Code']

# Step 3: Filter the DataFrame to include only those stations
filtered_df_2 = filtered_df[filtered_df["Station Code"].isin(valid_stations)]

In [41]:
print("Shape: ", filtered_df_2.shape)
print("Ilość stacji w odfiltrowanych meteo danych: ",filtered_df_2["Station Code"].nunique())

Shape:  (605780, 16)
Ilość stacji w odfiltrowanych meteo danych:  274


In [42]:
date_str = filtered_df_2['Year'].astype(str) + '-' + \
                        filtered_df_2['Month'].astype(str).str.zfill(2) + '-' + \
                        filtered_df_2['Day'].astype(str).str.zfill(2)
filtered_df_2['Calendar Date'] = pd.to_datetime(date_str, errors='coerce')

C:\Users\Tymek\AppData\Local\Temp\ipykernel_7976\3497555919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2['Calendar Date'] = pd.to_datetime(date_str, errors='coerce')


In [43]:
filtered_df_2

,Station Code,Station Name,Year,Month,Day,Daily Precip Sum,SMDB Measurement Status,Precipitation Type,Snow Cover Height,PKSN Measurement Status,Fresh Snow Height,HSS Measurement Status,Snow Type,GATS Measurement Status,Snow Cover Type,RPSN Measurement Status,Calendar Date
0,249180020,WARSZOWICE,2010,1,1,1.1,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-01
1,249180020,WARSZOWICE,2010,1,2,5.1,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-02
2,249180020,WARSZOWICE,2010,1,3,0.6,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-03
3,249180020,WARSZOWICE,2010,1,4,0.3,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-04
4,249180020,WARSZOWICE,2010,1,5,3.3,NaN,S,0,8.0,0,8.0,NaN,8.0,None,8.0,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13722,254220060,BANIE MAZURSKIE,2022,12,27,0.2,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-27
13723,254220060,BANIE MAZURSKIE,2022,12,28,5.1,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-28
13724,254220060,BANIE MAZURSKIE,2022,12,29,1.9,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-29
13725,254220060,BANIE MAZURSKIE,2022,12,30,0.3,NaN,W,0,9.0,0,9.0,NaN,9.0,.,NaN,2022-12-30


In [44]:
filtered_df_2.to_parquet("../static_data/filtered_meteo_data.parquet.gzip")